In [1]:
import os
import sys

import numpy as np
import pandas as pd

sys.path.insert(0, '..')

from app import db

pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 50)


<ipython-input-1-3ffd169b915e>:11: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [2]:
db_conn = db.get_db_conn()

## Get basic passing stats

In [3]:
query = """
    SELECT
        year,
        game_id,
        posteam AS team,
        defteam AS def_team,
        week,
        passer_id,
        passer,
        SUM(complete_pass) AS completions,
        SUM(pass_attempt) AS attempts,
        SUM(yards_gained) AS yards,
        SUM(air_yards) AS air_yards,
        SUM(pass_touchdown) AS td,
        SUM(interception) as int,
        SUM(fumble) as fumbles,
        SUM(epa) AS epa
    FROM
        play_by_play
    WHERE
        play_type = 'pass'
        AND two_point_attempt = 0
        AND sack = 0
    GROUP BY
        year, game_id, week, posteam, defteam, passer_id, passer
    ORDER BY
        yards DESC
"""
df_pass = pd.read_sql(query, db_conn)
print(df_pass.shape)
df_pass.head(10)

(636, 15)


,year,game_id,team,def_team,week,passer_id,passer,completions,attempts,yards,air_yards,td,int,fumbles,epa
0,2020,2020_04_CLE_DAL,DAL,CLE,4,32013030-2d30-3033-3330-37372d13d088,D.Prescott,41.0,58.0,502.0,491.0,4.0,1.0,0.0,20.550906
1,2020,2020_03_DAL_SEA,DAL,SEA,3,32013030-2d30-3033-3330-37372d13d088,D.Prescott,37.0,57.0,477.0,454.0,3.0,2.0,0.0,16.393521
2,2020,2020_12_KC_TB,KC,TB,12,32013030-2d30-3033-3338-3733fa30c4fa,P.Mahomes,37.0,49.0,462.0,485.0,3.0,0.0,0.0,27.965320
3,2020,2020_02_ATL_DAL,DAL,ATL,2,32013030-2d30-3033-3330-37372d13d088,D.Prescott,34.0,46.0,450.0,368.0,1.0,0.0,1.0,17.193913
4,2020,2020_01_SEA_ATL,ATL,SEA,1,32013030-2d30-3032-3631-34338634c9b6,M.Ryan,37.0,54.0,450.0,507.0,2.0,1.0,0.0,12.932285
5,2020,2020_08_NYJ_KC,KC,NYJ,8,32013030-2d30-3033-3338-3733fa30c4fa,P.Mahomes,31.0,42.0,416.0,388.0,5.0,0.0,1.0,27.690373
6,2020,2020_02_BUF_MIA,BUF,MIA,2,32013030-2d30-3033-3438-3537f3a13a66,J.Allen,24.0,35.0,415.0,381.0,4.0,0.0,1.0,24.456950
7,2020,2020_09_SEA_BUF,BUF,SEA,9,32013030-2d30-3033-3438-3537f3a13a66,J.Allen,31.0,38.0,415.0,341.0,3.0,0.0,0.0,24.622421
8,2020,2020_15_PHI_ARI,ARI,PHI,15,32013030-2d30-3033-3532-323871689b29,K.Murray,27.0,36.0,406.0,350.0,3.0,1.0,1.0,8.441423
9,2020,2020_07_CLE_CIN,CIN,CLE,7,32013030-2d30-3033-3634-3432d92fc82e,J.Burrow,35.0,47.0,406.0,462.0,3.0,1.0,1.0,27.058722


## Debugging

In [4]:
query = """
    SELECT
        year,
        game_id,
        posteam AS team,
        defteam AS def_team,
        week,
        passer_id,
        passer,
        complete_pass AS completions,
        pass_attempt AS attempts,
        yards_gained AS yards,
        air_yards AS air_yards,
        pass_touchdown AS td,
        interception as int,
        epa AS epa
    FROM
        play_by_play
    WHERE
        play_type = 'pass'
        AND two_point_attempt = 0
        -- AND sack = 0
        AND complete_pass = 0
        AND incomplete_pass = 0
        AND interception = 0
"""
df = pd.read_sql(query, db_conn)
print(df.shape)
df.head()

(1135, 14)


,year,game_id,team,def_team,week,passer_id,passer,completions,attempts,yards,air_yards,td,int,epa
0,2020,2020_01_ARI_SF,SF,ARI,1,32013030-2d30-3033-3133-34353afe165e,J.Garoppolo,0.0,1.0,-7.0,None,0.0,0.0,-1.607617
1,2020,2020_01_ARI_SF,SF,ARI,1,32013030-2d30-3033-3133-34353afe165e,J.Garoppolo,0.0,1.0,-5.0,None,0.0,0.0,-1.222099
2,2020,2020_01_ARI_SF,ARI,SF,1,32013030-2d30-3033-3532-323871689b29,K.Murray,0.0,1.0,-5.0,None,0.0,0.0,-1.629293
3,2020,2020_01_ARI_SF,SF,ARI,1,32013030-2d30-3033-3133-34353afe165e,J.Garoppolo,0.0,1.0,-4.0,None,0.0,0.0,-1.392170
4,2020,2020_01_ARI_SF,ARI,SF,1,32013030-2d30-3033-3532-323871689b29,K.Murray,0.0,1.0,-1.0,None,0.0,0.0,-0.780837


In [5]:
query = """
    SELECT
        year,
        game_id,
        posteam AS team,
        defteam AS def_team,
        week,
        passer_id,
        passer,
        complete_pass AS completions,
        pass_attempt AS attempts,
        yards_gained AS yards,
        air_yards AS air_yards,
        pass_touchdown AS td,
        interception as int,
        epa AS epa
    FROM
        play_by_play
    WHERE
        play_type = 'pass'
        -- AND two_point_attempt = 0
        -- AND sack = 0
        AND complete_pass = 1
        AND incomplete_pass = 1
"""
df = pd.read_sql(query, db_conn)
print(df.shape)
df.head()

(0, 14)


,year,game_id,team,def_team,week,passer_id,passer,completions,attempts,yards,air_yards,td,int,epa


In [6]:
query = """
    SELECT
        year,
        game_id,
        posteam AS team,
        defteam AS def_team,
        week,
        passer_id,
        passer,
        complete_pass AS completions,
        pass_attempt AS attempts,
        yards_gained AS yards,
        air_yards AS air_yards,
        pass_touchdown AS td,
        interception as int,
        epa AS epa
    FROM
        play_by_play
    WHERE
        play_type = 'pass'
        AND two_point_attempt = 0
        AND sack = 0
        AND yards_gained > 0
        AND incomplete_pass = 1
"""
df = pd.read_sql(query, db_conn)
print(df.shape)
df.head()

(0, 14)


,year,game_id,team,def_team,week,passer_id,passer,completions,attempts,yards,air_yards,td,int,epa


In [7]:
query = """
    SELECT
        year,
        game_id,
        posteam AS team,
        defteam AS def_team,
        week,
        passer,
        SUM(complete_pass) AS completions,
        SUM(incomplete_pass) AS incompletions,
        SUM(pass_attempt) AS attempts,
        SUM(yards_gained) AS yards,
        SUM(air_yards) AS air_yards,
        SUM(pass_touchdown) AS td,
        SUM(interception) as int,
        SUM(fumble) as fumbles,
        SUM(epa) AS epa
    FROM
        play_by_play
    WHERE
        play_type = 'pass'
        AND two_point_attempt = 0
        AND sack = 0
        AND passer = 'A.Rodgers'
    GROUP BY
        year, game_id, week, posteam, defteam, passer
    ORDER BY
        week DESC
"""
df_pass = pd.read_sql(query, db_conn)
print(df_pass.shape)
df_pass.head(20)

(16, 15)


,year,game_id,team,def_team,week,passer,completions,incompletions,attempts,yards,air_yards,td,int,fumbles,epa
0,2020,2020_17_GB_CHI,GB,CHI,17,A.Rodgers,19.0,5.0,24.0,240.0,159.0,4.0,0.0,0.0,22.304879
1,2020,2020_16_TEN_GB,GB,TEN,16,A.Rodgers,21.0,3.0,25.0,231.0,178.0,4.0,1.0,0.0,13.729254
2,2020,2020_15_CAR_GB,GB,CAR,15,A.Rodgers,20.0,9.0,29.0,143.0,101.0,1.0,0.0,0.0,-0.114895
3,2020,2020_14_GB_DET,GB,DET,14,A.Rodgers,26.0,7.0,33.0,290.0,210.0,3.0,0.0,0.0,17.861166
4,2020,2020_13_PHI_GB,GB,PHI,13,A.Rodgers,25.0,9.0,34.0,295.0,364.0,3.0,0.0,0.0,17.272169
5,2020,2020_12_CHI_GB,GB,CHI,12,A.Rodgers,21.0,8.0,29.0,211.0,244.0,4.0,0.0,0.0,15.347127
6,2020,2020_11_GB_IND,GB,IND,11,A.Rodgers,27.0,8.0,36.0,311.0,297.0,3.0,1.0,1.0,3.810286
7,2020,2020_10_JAX_GB,GB,JAX,10,A.Rodgers,24.0,9.0,34.0,325.0,206.0,2.0,1.0,2.0,2.024708
8,2020,2020_09_GB_SF,GB,SF,9,A.Rodgers,25.0,6.0,31.0,305.0,224.0,4.0,0.0,0.0,20.081083
9,2020,2020_08_MIN_GB,GB,MIN,8,A.Rodgers,27.0,13.0,40.0,291.0,311.0,3.0,0.0,0.0,12.170511


In [8]:
query = """
    SELECT
        year,
        game_id,
        posteam AS team,
        defteam AS def_team,
        week,
        passer,
        receiver,
        SUM(complete_pass) AS completions,
        SUM(incomplete_pass) AS incompletions,
        SUM(pass_attempt) AS attempts,
        SUM(yards_gained) AS yards,
        SUM(air_yards) AS air_yards,
        SUM(pass_touchdown) AS td,
        SUM(interception) as int,
        SUM(fumble) as fumbles,
        SUM(epa) AS epa
    FROM
        play_by_play
    WHERE
        play_type = 'pass'
        AND two_point_attempt = 0
        AND sack = 0
        AND passer = 'A.Rodgers'
        AND game_id = '2020_11_GB_IND'
    GROUP BY
        year, game_id, week, posteam, defteam, passer, receiver
    ORDER BY
        yards DESC
"""
df_pass = pd.read_sql(query, db_conn)
print(df_pass.shape)
df_pass.head(20)

(10, 16)


,year,game_id,team,def_team,week,passer,receiver,completions,incompletions,attempts,yards,air_yards,td,int,fumbles,epa
0,2020,2020_11_GB_IND,GB,IND,11,A.Rodgers,D.Adams,7.0,2.0,9.0,106.0,86.0,1.0,0.0,0.0,5.162734
1,2020,2020_11_GB_IND,GB,IND,11,A.Rodgers,M.Valdes-Scantling,3.0,2.0,6.0,55.0,114.0,0.0,1.0,1.0,-7.583106
2,2020,2020_11_GB_IND,GB,IND,11,A.Rodgers,R.Tonyan,5.0,0.0,5.0,44.0,35.0,1.0,0.0,0.0,1.962793
3,2020,2020_11_GB_IND,GB,IND,11,A.Rodgers,A.Jones,4.0,0.0,4.0,30.0,1.0,0.0,0.0,0.0,1.603021
4,2020,2020_11_GB_IND,GB,IND,11,A.Rodgers,E.St,1.0,0.0,1.0,23.0,-2.0,0.0,0.0,0.0,1.787997
5,2020,2020_11_GB_IND,GB,IND,11,A.Rodgers,J.Sternberger,3.0,0.0,3.0,23.0,6.0,0.0,0.0,0.0,1.227790
6,2020,2020_11_GB_IND,GB,IND,11,A.Rodgers,A.Lazard,2.0,2.0,4.0,18.0,35.0,0.0,0.0,0.0,0.552374
7,2020,2020_11_GB_IND,GB,IND,11,A.Rodgers,M.Lewis,1.0,0.0,1.0,8.0,5.0,0.0,0.0,0.0,1.184768
8,2020,2020_11_GB_IND,GB,IND,11,A.Rodgers,J.Williams,1.0,1.0,2.0,4.0,8.0,1.0,0.0,0.0,-1.870769
9,2020,2020_11_GB_IND,GB,IND,11,A.Rodgers,None,0.0,1.0,1.0,0.0,9.0,0.0,0.0,0.0,-0.217315


### WHY IS AROD SHORT 2 ATTEMPTS???

In [9]:
query = """
    SELECT
        year,
        game_id,
        play_type,
        posteam AS team,
        pass_attempt,
        complete_pass,
        incomplete_pass,
        sack,
        week,
        passer,
        receiver,
        "desc"
    FROM
        play_by_play
    WHERE
        "desc" LIKE '%A.Rodgers%'
        -- passer = 'A.Rodgers' 
        -- AND play_type = 'pass' OR play_type = 'no'
        AND two_point_attempt = 0
        -- AND sack = 0
        AND game_id = '2020_11_GB_IND'
    ORDER BY play_type
"""
df_pass = pd.read_sql(query, db_conn)
print(df_pass.shape)
df_pass.head(len(df_pass))

(43, 12)


,year,game_id,play_type,team,pass_attempt,complete_pass,incomplete_pass,sack,week,passer,receiver,desc
0,2020,2020_11_GB_IND,no_play,GB,0.0,0.0,0.0,0.0,11,A.Rodgers,M.Valdes-Scantling,"(:27) (Shotgun) 12-A.Rodgers pass incomplete deep middle to 83-M.Valdes-Scantling (26-R.Ya-Sin). PENALTY on IND-26-R.Ya-Sin, Defensive Pass Interference, 51 yards, enforced at GB 45 - No Play."
1,2020,2020_11_GB_IND,pass,GB,1.0,1.0,0.0,0.0,11,A.Rodgers,A.Lazard,(14:17) (Shotgun) 12-A.Rodgers pass short right to 13-A.Lazard pushed ob at IND 38 for 8 yards (26-R.Ya-Sin).
2,2020,2020_11_GB_IND,pass,GB,1.0,1.0,0.0,0.0,11,A.Rodgers,M.Valdes-Scantling,"(9:20) (Shotgun) 12-A.Rodgers pass short left to 83-M.Valdes-Scantling to GB 30 for no gain (32-J.Blackmon). FUMBLES (32-J.Blackmon), RECOVERED by IND-99-D.Buckner at GB 29."
3,2020,2020_11_GB_IND,pass,GB,1.0,1.0,0.0,0.0,11,A.Rodgers,D.Adams,(:57) (Shotgun) 12-A.Rodgers pass short middle to 17-D.Adams to IND 15 for 18 yards (23-K.Moore).
4,2020,2020_11_GB_IND,pass,GB,1.0,1.0,0.0,0.0,11,A.Rodgers,R.Tonyan,(:42) (Shotgun) 12-A.Rodgers pass short middle to 85-R.Tonyan to IND 8 for 7 yards (23-K.Moore).
5,2020,2020_11_GB_IND,pass,GB,1.0,0.0,1.0,0.0,11,A.Rodgers,D.Adams,"(:12) (No Huddle, Shotgun) 12-A.Rodgers pass incomplete short left to 17-D.Adams."
6,2020,2020_11_GB_IND,pass,GB,1.0,1.0,0.0,0.0,11,A.Rodgers,A.Jones,"(9:56) (Shotgun) 12-A.Rodgers pass short right to 33-A.Jones to GB 30 for 8 yards (54-A.Walker, 53-D.Leonard). IND-54-A.Walker was injured during the play."
7,2020,2020_11_GB_IND,pass,GB,1.0,1.0,0.0,0.0,11,A.Rodgers,D.Adams,(14:55) (Shotgun) 12-A.Rodgers pass deep right to 17-D.Adams pushed ob at IND 46 for 33 yards (26-R.Ya-Sin).
8,2020,2020_11_GB_IND,pass,GB,1.0,1.0,0.0,0.0,11,A.Rodgers,R.Tonyan,(10:29) 12-A.Rodgers pass short right to 85-R.Tonyan to IND 34 for 4 yards (26-R.Ya-Sin).
9,2020,2020_11_GB_IND,pass,GB,1.0,1.0,0.0,0.0,11,A.Rodgers,M.Lewis,(9:48) 12-A.Rodgers pass short left to 89-M.Lewis to IND 26 for 8 yards (23-K.Moore).


## Takeaway:
need to count qb_spike!! When their is a qb spike, passer is None but passer_palyer_name isn't, so we need to group on that value. We also can't group by passer_id, so we need to join that back in.

## Passing by player by year

In [10]:
query = """
    SELECT
        year,
        posteam AS team,
        passer_player_name,
        SUM(complete_pass) AS completions,
        SUM(pass_attempt) AS attempts,
        SUM(yards_gained) AS yards,
        SUM(air_yards) AS air_yards,
        SUM(pass_touchdown) AS td,
        SUM(interception) as int,
        SUM(fumble) as fumbles,
        SUM(epa) AS epa
    FROM
        play_by_play
    WHERE
        (play_type = 'pass' or play_type = 'qb_spike')
        AND two_point_attempt = 0
        AND sack = 0
    GROUP BY 
        year, posteam, passer_player_name
    ORDER BY
        yards DESC
"""
df_pass = pd.read_sql(query, db_conn)
print(df_pass.shape)
df_pass.head(10)

(114, 11)


,year,team,passer_player_name,completions,attempts,yards,air_yards,td,int,fumbles,epa
0,2020,HOU,D.Watson,382.0,544.0,4801.0,4821.0,33.0,7.0,7.0,195.740604
1,2020,KC,P.Mahomes,390.0,588.0,4740.0,4750.0,38.0,6.0,8.0,205.672649
2,2020,TB,T.Brady,401.0,610.0,4633.0,5539.0,40.0,12.0,6.0,153.866315
3,2020,ATL,M.Ryan,407.0,626.0,4581.0,5262.0,26.0,11.0,5.0,128.209560
4,2020,BUF,J.Allen,396.0,572.0,4544.0,4880.0,37.0,10.0,3.0,202.917284
5,2020,LAC,J.Herbert,396.0,595.0,4336.0,4386.0,31.0,10.0,4.0,140.435692
6,2020,GB,A.Rodgers,372.0,526.0,4299.0,4066.0,48.0,5.0,3.0,207.913831
7,2020,MIN,K.Cousins,349.0,516.0,4265.0,4071.0,35.0,13.0,4.0,155.710488
8,2020,SEA,R.Wilson,384.0,558.0,4212.0,4824.0,40.0,13.0,3.0,150.432104
9,2020,IND,P.Rivers,369.0,543.0,4169.0,3883.0,24.0,11.0,4.0,125.740980


### Ok, I'm slightly undercounting attempts...

In [16]:
query = """
    SELECT
        year,
        posteam AS team,
        passer,
        -- passer_player_name,
        SUM(complete_pass) AS completions,
        SUM(pass_attempt) AS attempts,
        SUM(yards_gained) AS yards,
        SUM(air_yards) AS air_yards,
        SUM(pass_touchdown) AS td,
        SUM(interception) as int,
        SUM(fumble) as fumbles,
        SUM(epa) AS epa
    FROM
        play_by_play_enriched
    WHERE
        (play_type = 'pass' or play_type = 'qb_spike')
        AND two_point_attempt = 0
        AND sack = 0
        AND (passer = 'J.Herbert' or passer_player_name = 'J.Herbert')
        
    GROUP BY 
        year, posteam, passer --, passer_player_name
    ORDER BY
        yards DESC
"""
df_pass = pd.read_sql(query, db_conn)
print(df_pass.shape)
df_pass.head(20)

(1, 11)


,year,team,passer,completions,attempts,yards,air_yards,td,int,fumbles,epa
0,2020,LAC,J.Herbert,396.0,595.0,4336.0,4386.0,31.0,10.0,4.0,140.435692


## passing_by_player_by_year


In [22]:
query = """
    SELECT
        year,
        posteam AS team,
        passer,
        COUNT(DISTINCT week) as games,
        SUM(complete_pass) AS completions,
        SUM(pass_attempt) AS attempts,
        SUM(yards_gained) AS yards,
        SUM(air_yards) AS air_yards_intended,
        SUM(CASE WHEN complete_pass = 1 THEN air_yards ELSE 0 END) AS air_yards_completed,
        SUM(pass_touchdown) AS td,
        SUM(interception) as int,
        SUM(fumble) as fumbles,
        SUM(epa) AS epa
    FROM
        play_by_play_enriched
    WHERE
        (play_type = 'pass' or play_type = 'qb_spike')
        AND two_point_attempt = 0
        AND sack = 0
    GROUP BY
        year, posteam, passer  
    ORDER BY
        yards DESC
"""
df_pass = pd.read_sql(query, db_conn)
print(df_pass.shape)
df_pass.head(10)

(112, 13)


,year,team,passer,games,completions,attempts,yards,air_yards_intended,air_yards_completed,td,int,fumbles,epa
0,2020,HOU,D.Watson,16,382.0,544.0,4801.0,4821.0,2856.0,33.0,7.0,7.0,195.740604
1,2020,KC,P.Mahomes,15,390.0,588.0,4740.0,4750.0,2481.0,38.0,6.0,8.0,205.672649
2,2020,TB,T.Brady,16,401.0,610.0,4633.0,5539.0,2839.0,40.0,12.0,6.0,153.866315
3,2020,ATL,M.Ryan,16,407.0,626.0,4581.0,5262.0,2913.0,26.0,11.0,5.0,128.209560
4,2020,BUF,J.Allen,16,396.0,572.0,4544.0,4880.0,2696.0,37.0,10.0,3.0,202.917284
5,2020,LAC,J.Herbert,15,396.0,595.0,4336.0,4386.0,2129.0,31.0,10.0,4.0,140.435692
6,2020,GB,A.Rodgers,16,372.0,526.0,4299.0,4066.0,2027.0,48.0,5.0,3.0,207.913831
7,2020,MIN,K.Cousins,16,349.0,516.0,4265.0,4071.0,2312.0,35.0,13.0,4.0,155.710488
8,2020,SEA,R.Wilson,16,384.0,558.0,4212.0,4824.0,2400.0,40.0,13.0,3.0,150.432104
9,2020,IND,P.Rivers,16,369.0,543.0,4169.0,3883.0,1948.0,24.0,11.0,4.0,125.740980


## passing_by_player_by_game

In [29]:
## Deshaun is missing some yards - looks like week 16 at cinci is the culprint
query = """
    SELECT
        year,
        posteam AS team,
        passer,
        week,
        defteam as opp,
        SUM(complete_pass) AS completions,
        SUM(pass_attempt) AS attempts,
        SUM(yards_gained) AS yards,
        SUM(air_yards) AS air_yards_intended,
        SUM(CASE WHEN complete_pass = 1 THEN air_yards ELSE 0 END) AS air_yards_completed,
        SUM(pass_touchdown) AS td,
        SUM(interception) as int,
        SUM(fumble) as fumbles,
        SUM(epa) AS epa
    FROM
        play_by_play_enriched
    WHERE
        (play_type = 'pass' or play_type = 'qb_spike')
        AND two_point_attempt = 0
        AND sack = 0
        AND passer = 'D.Watson'
    GROUP BY
        year, week, defteam, posteam, passer  
    ORDER BY 
        week DESC
"""
df_pass = pd.read_sql(query, db_conn)
print(df_pass.shape)
df_pass.head(10)

(16, 14)


,year,team,passer,week,opp,completions,attempts,yards,air_yards_intended,air_yards_completed,td,int,fumbles,epa
0,2020,HOU,D.Watson,17,TEN,28.0,39.0,365.0,398.0,272.0,3.0,1.0,0.0,14.918754
1,2020,HOU,D.Watson,16,CIN,24.0,33.0,302.0,339.0,186.0,3.0,0.0,2.0,8.704536
2,2020,HOU,D.Watson,15,IND,33.0,41.0,373.0,313.0,233.0,2.0,0.0,2.0,19.259301
3,2020,HOU,D.Watson,14,CHI,21.0,30.0,219.0,200.0,100.0,1.0,0.0,0.0,9.912673
4,2020,HOU,D.Watson,13,IND,26.0,38.0,341.0,394.0,235.0,0.0,1.0,0.0,8.053666
5,2020,HOU,D.Watson,12,DET,17.0,25.0,318.0,319.0,203.0,4.0,0.0,0.0,21.294212
6,2020,HOU,D.Watson,11,NE,28.0,37.0,344.0,314.0,177.0,2.0,0.0,0.0,20.124489
7,2020,HOU,D.Watson,10,CLE,20.0,30.0,163.0,180.0,71.0,1.0,0.0,0.0,4.125785
8,2020,HOU,D.Watson,9,JAX,19.0,32.0,281.0,291.0,87.0,2.0,0.0,0.0,10.497332
9,2020,HOU,D.Watson,7,GB,29.0,39.0,309.0,248.0,176.0,2.0,0.0,2.0,9.810870


## Deshaun missing some yards...
Watson cooks missing 22 yards and Charlie heck has 0 receptions for 8 yards on ESPN??
Cooks missing 7 yards, Heck missing 8, Scharping missing 7


In [32]:
## Deshaun is missing some yards - looks like week 16 at cinci is the culprint
query = """
    SELECT
        year,
        posteam AS team,
        week,
        defteam as opp,
        passer,
        receiver,
        SUM(complete_pass) AS completions,
        SUM(pass_attempt) AS attempts,
        SUM(yards_gained) AS yards,
        SUM(air_yards) AS air_yards_intended,
        SUM(CASE WHEN complete_pass = 1 THEN air_yards ELSE 0 END) AS air_yards_completed,
        SUM(pass_touchdown) AS td,
        SUM(interception) as int,
        SUM(fumble) as fumbles,
        SUM(epa) AS epa
    FROM
        play_by_play_enriched
    WHERE
        (play_type = 'pass' or play_type = 'qb_spike')
        AND two_point_attempt = 0
        AND sack = 0
        AND passer = 'D.Watson'
        AND week = 16
    GROUP BY
        year, week, defteam, posteam, passer, receiver 
    ORDER BY 
        yards DESC
"""
df_pass = pd.read_sql(query, db_conn)
print(df_pass.shape)
df_pass.head(10)

(9, 15)


,year,team,week,opp,passer,receiver,completions,attempts,yards,air_yards_intended,air_yards_completed,td,int,fumbles,epa
0,2020,HOU,16,CIN,D.Watson,B.Cooks,7.0,10.0,134.0,173.0,104.0,1.0,0.0,0.0,6.607903
1,2020,HOU,16,CIN,D.Watson,K.Coutee,5.0,5.0,54.0,36.0,36.0,0.0,0.0,1.0,2.510797
2,2020,HOU,16,CIN,D.Watson,J.Akins,4.0,4.0,39.0,19.0,19.0,0.0,0.0,0.0,2.196513
3,2020,HOU,16,CIN,D.Watson,P.Brown,1.0,1.0,29.0,8.0,8.0,0.0,0.0,0.0,1.234720
4,2020,HOU,16,CIN,D.Watson,D.Fells,1.0,2.0,22.0,25.0,11.0,1.0,0.0,0.0,2.349711
5,2020,HOU,16,CIN,D.Watson,D.Johnson,3.0,3.0,11.0,-7.0,-7.0,1.0,0.0,0.0,0.819004
6,2020,HOU,16,CIN,D.Watson,C.Hansen,1.0,3.0,8.0,47.0,11.0,0.0,0.0,1.0,-2.273367
7,2020,HOU,16,CIN,D.Watson,K.Warring,1.0,3.0,3.0,11.0,2.0,0.0,0.0,0.0,-3.723865
8,2020,HOU,16,CIN,D.Watson,S.Mitchell,1.0,2.0,2.0,27.0,2.0,0.0,0.0,0.0,-1.016879


In [44]:
## OK we know there was a lateral on this bitch that gives some weird yards
query = """
    SELECT
       *
    FROM
        play_by_play_enriched
    WHERE
    week = 16 AND
        "desc" LIKE '%Scharping%'

"""
df_pass = pd.read_sql(query, db_conn)
print(df_pass.shape)
df_pass.head(10)

(2, 343)


,play_id,game_id,old_game_id,home_team,away_team,season_type,week,posteam,posteam_type,defteam,side_of_field,yardline_100,game_date,quarter_seconds_remaining,half_seconds_remaining,game_seconds_remaining,game_half,quarter_end,drive,sp,qtr,down,goal_to_go,time,yrdln,ydstogo,ydsnet,desc,play_type,yards_gained,shotgun,no_huddle,qb_dropback,qb_kneel,qb_spike,qb_scramble,pass_length,pass_location,air_yards,yards_after_catch,run_location,run_gap,field_goal_result,kick_distance,extra_point_result,two_point_conv_result,home_timeouts_remaining,away_timeouts_remaining,timeout,timeout_team,td_team,posteam_timeouts_remaining,defteam_timeouts_remaining,total_home_score,total_away_score,posteam_score,defteam_score,score_differential,posteam_score_post,defteam_score_post,score_differential_post,no_score_prob,opp_fg_prob,opp_safety_prob,opp_td_prob,fg_prob,safety_prob,td_prob,extra_point_prob,two_point_conversion_prob,ep,epa,total_home_epa,total_away_epa,total_home_rush_epa,total_away_rush_epa,total_home_pass_epa,total_away_pass_epa,air_epa,yac_epa,comp_air_epa,comp_yac_epa,total_home_comp_air_epa,total_away_comp_air_epa,total_home_comp_yac_epa,total_away_comp_yac_epa,total_home_raw_air_epa,total_away_raw_air_epa,total_home_raw_yac_epa,total_away_raw_yac_epa,wp,def_wp,home_wp,away_wp,wpa,home_wp_post,away_wp_post,vegas_wp,vegas_home_wp,total_home_rush_wpa,...,fumble_recovery_1_player_name,fumble_recovery_2_team,fumble_recovery_2_yards,fumble_recovery_2_player_id,fumble_recovery_2_player_name,return_team,return_yards,penalty_team,penalty_player_id,penalty_player_name,penalty_yards,replay_or_challenge,replay_or_challenge_result,penalty_type,defensive_two_point_attempt,defensive_two_point_conv,defensive_extra_point_attempt,defensive_extra_point_conv,season,cp,cpoe,series,series_success,series_result,order_sequence,start_time,time_of_day,stadium,weather,nfl_api_id,play_clock,play_deleted,play_type_nfl,special_teams_play,st_play_type,end_clock_time,end_yard_line,fixed_drive,fixed_drive_result,drive_real_start_time,drive_play_count,drive_time_of_possession,drive_first_downs,drive_inside20,drive_ended_with_score,drive_quarter_start,drive_quarter_end,drive_yards_penalized,drive_start_transition,drive_end_transition,drive_game_clock_start,drive_game_clock_end,drive_start_yard_line,drive_end_yard_line,drive_play_id_started,drive_play_id_ended,away_score,home_score,location,result,total,spread_line,total_line,div_game,roof,surface,temp,wind,home_coach,away_coach,stadium_id,game_stadium,success,passer,passer_jersey_number,rusher,rusher_jersey_number,receiver,receiver_jersey_number,pass,rush,first_down,aborted_play,special,play,passer_id,rusher_id,receiver_id,name,jersey_number,id,qb_epa,xyac_epa,xyac_mean_yardage,xyac_median_yardage,xyac_success,xyac_fd,year,gsis_id,position
0,2600,2020_16_CIN_HOU,2020122706,HOU,CIN,REG,16,HOU,home,CIN,CIN,3.0,2020-12-27,170.0,1070.0,1070.0,Half2,0,16.0,0,3,1.0,1,02:50,CIN 3,3,75.0,"(2:50) (Shotgun) PENALTY on HOU-74-M.Scharping, False Start, 5 yards, enforced at CIN 3 - No Play.",no_play,0.0,1,0,0.0,0,0,0,None,None,NaN,NaN,None,None,None,None,None,None,3,3,0.0,None,None,3.0,3.0,17,24,17.0,24.0,-7.0,17.0,24.0,-7.0,0.002067,0.008600,0.000278,0.010102,0.121406,0.000662,0.856884,0.0,0.0,6.266665,-0.636102,-4.849149,4.849149,6.981428,-6.981428,-14.928389,14.928389,NaN,NaN,0.000000,0.000000,0.183946,-0.183946,-14.684971,14.684971,2.138482,-2.138482,-17.06687,17.06687,0.480916,0.519084,0.480916,0.519084,-0.033319,0.447597,0.552403,0.562792,0.562792,0.235788,...,None,None,None,None,None,None,0.0,HOU,32013030-2d30-3033-3532-343595c3ab30,M.Scharping,5.0,0,None,False Start,0.0,0.0,0.0,0.0,2020,NaN,NaN,41,1,Touchdown,2600.0,13:00:00,20:10:07,NRG Stadium,"Sunny Temp: 70° F, Humidity: 80%, Wind: SSE 12 mph",10160000-0583-99cf-acec-c51ee1a30702,3,0,PENALTY,0,None,None,CIN 8,16,Touchdown,None,8.0,4:51,4.0,1.0,1.0,3.0,3.0,-5.0,KICKOFF,TOUCHDOWN,05:46,00:55,HOU 25,CIN 2,2462.0,2702.0,37,31,Home,-6,68,7.5,46.0,0,None,None,None,None,Romeo 

In [39]:
## Deshaun is missing some yards - looks like week 16 at cinci is the culprint
query = """
    SELECT
        year,
        posteam AS team,
        week,
        defteam as opp,
        passer,
        receiver,
        "desc"
    FROM
        play_by_play_enriched
    WHERE
        (play_type = 'pass' or play_type = 'qb_spike')
        AND two_point_attempt = 0
        AND sack = 0
        AND passer = 'D.Watson'
        AND week = 16 

"""
df_pass = pd.read_sql(query, db_conn)
print(df_pass.shape)
df_pass.head(10)

(33, 7)


,year,team,week,opp,passer,receiver,desc
0,2020,HOU,16,CIN,D.Watson,B.Cooks,(15:00) 4-D.Watson pass incomplete short middle to 13-B.Cooks (23-D.Phillips).
1,2020,HOU,16,CIN,D.Watson,K.Warring,(14:19) (Shotgun) 4-D.Watson pass short right to 81-K.Warring to HOU 31 for 3 yards (24-V.Bell; 23-D.Phillips).
2,2020,HOU,16,CIN,D.Watson,B.Cooks,(8:32) 4-D.Watson pass short right to 13-B.Cooks to HOU 32 for 7 yards (23-D.Phillips).
3,2020,HOU,16,CIN,D.Watson,C.Hansen,(7:19) (Shotgun) 4-D.Watson pass incomplete short left to 17-C.Hansen.
4,2020,HOU,16,CIN,D.Watson,B.Cooks,(3:33) 4-D.Watson pass deep right to 13-B.Cooks to CIN 7 for 50 yards (22-W.Jackson III).
5,2020,HOU,16,CIN,D.Watson,S.Mitchell,(2:43) (Shotgun) 4-D.Watson pass short middle to 11-S.Mitchell to CIN 5 for 2 yards (56-J.Bynes).
6,2020,HOU,16,CIN,D.Watson,K.Warring,(1:29) (Shotgun) 4-D.Watson pass incomplete short right to 81-K.Warring.
7,2020,HOU,16,CIN,D.Watson,K.Warring,(8:28) (Shotgun) 4-D.Watson pass incomplete short middle to 81-K.Warring (23-D.Phillips).
8,2020,HOU,16,CIN,D.Watson,D.Johnson,(8:25) (Shotgun) 4-D.Watson pass short right to 31-D.Johnson to CIN 42 for 12 yards (56-J.Bynes). HOU-76-B.Qvale was injured during the play. He is Out.
9,2020,HOU,16,CIN,D.Watson,B.Cooks,(7:51) (Shotgun) 4-D.Watson pass incomplete short middle to 13-B.Cooks (51-M.Bailey) [58-C.Lawson].


In [184]:
caption_props = [
    ('font-size', '28px'),
    ('text-align', 'center'),
    ('color', 'black'),
    ('font-family', 'Trebuchet MS'),
    #('font-family', 'Courier New'),
    ('font-weight', '560')
    
]

th_props = [
    ('font-size', '13px'),
    ('text-align', 'center'),
    ('font-weight', 'bold'),
    ('font-family', 'Trebuchet MS')
  # ('color', '#6d6d6d'),
  # ('background-color', '#f7f7f9')
]

# Set CSS properties for td elements in dataframe
td_props = [
    ('font-size', '14px'),
    ('font-weight', '100'),
    #('font-family', 'Courier New'),
    ('font-family', 'Tahoma'),
    ('letter-spacing', '.4px')
]

name_props = [
   ('font-size', '14px'),
#     ('font-weight', '500'),
#     ('font-family', 'Courier New'),
    ('letter-spacing', '.8px')
]

highlight_props = [
    # ('font-size', '14px'),
    ('font-weight', '600'),
    # ('font-family', 'Courier New'),

]

styles = [
    dict(selector="th", props=th_props),
    dict(selector="td", props=td_props),
    dict(selector="caption", props=caption_props),
    dict(selector="td:nth-child(2)", props=name_props),
    dict(selector="td:nth-child(7)", props=highlight_props),
]

df_styled = df_pass.style \
    .format({
        "td": "{:.1f}",
        "int": "{:.2f}"
    }) \
    .background_gradient(subset=['td', 'yards'], cmap='Greens') \
    .set_caption('Week 1 QB Report') \
    .set_table_styles(styles)

df_styled

,year,team,passer_player_name,completions,attempts,yards,air_yards_intended,air_yards_completed,td,int,fumbles,epa
0,2020,KC,P.Mahomes,340.000000,497.000000,4208.000000,4024.000000,2243.000000,33.0,5.00,7.000000,197.366213
1,2020,LAC,J.Herbert,353.000000,531.000000,3781.000000,3935.000000,1876.000000,27.0,10.00,4.000000,115.134112
2,2020,HOU,D.Watson,297.000000,431.000000,3761.000000,3771.000000,2165.000000,25.0,6.00,3.000000,152.857831
3,2020,GB,A.Rodgers,312.000000,448.000000,3685.000000,3628.000000,1794.000000,39.0,4.00,3.000000,171.994463
4,2020,SEA,R.Wilson,326.000000,463.000000,3685.000000,4043.000000,2128.000000,36.0,12.00,3.000000,141.078403
5,2020,ATL,M.Ryan,317.000000,498.000000,3660.000000,4376.000000,2386.000000,19.0,11.00,1.000000,99.162026
6,2020,BUF,J.Allen,323.000000,471.000000,3641.000000,3871.000000,2112.000000,28.0,9.00,2.000000,144.174632
7,2020,DET,M.Stafford,295.000000,462.000000,3522.000000,3859.000000,1975.000000,22.0,9.00,1.000000,90.214376
8,2020,LA,J.Goff,324.000000,475.000000,3509.000000,3120.000000,1564.000000,18.0,11.00,4.000000,80.288641
9,2020,IND,P.Rivers,309.000000,454.000000,3507.000000,3160.000000,1637.000000,20.0,9.00,4.000000,107.240602


In [180]:
query = """
    SELECT DISTINCT passer, passer_id
    FROM play_by_play
"""
df_pass = pd.read_sql(query, db_conn)
print(df_pass.shape)
df_pass.sort_values('passer').head(50)

(106, 2)


,passer,passer_id
21,A.Dalton,32013030-2d30-3032-3739-37334d29528c
72,A.Humphries,32013030-2d30-3033-3230-30396f0c2958
103,A.McCarron,32013030-2d30-3033-3132-38386b3121bc
12,A.Rodgers,32013030-2d30-3032-3334-35395dc60da5
54,A.Smith,32013030-2d30-3032-3334-3336b638d37d
31,B.Allen,32013030-2d30-3033-3234-33349fe923c0
30,B.Colquitt,32013030-2d30-3032-3635-31361c4d7f0a
76,B.DiNucci,32013030-2d30-3033-3633-38343a2e65ef
59,B.Gabbert,32013030-2d30-3032-3739-34386d7e96d3
74,B.Hoyer,32013030-2d30-3032-3636-3235a601d8e5
